## Prolog Tool

The PrologTool class allows the generation of langchain tools that use Prolog rules to generate answers.

Let's use the following Prolog rules in the file family.pl:

parent(john, bianca, mary).\
parent(john, bianca, michael).\
parent(peter, patricia, jennifer).\
partner(X, Y) :- parent(X, Y, _).

### Instantiation

The most important classes in langchain-prolog are: `PrologConfig`, `PrologRunnable` and `PrologTool`
- `PrologConfig` sets the configuration for the Prolog interpreter. The only mandatory field is the path to the Prolog script to be used.
- `PrologRunnable.create_schema` defines a Pydantic schema to be used to pass arguments to the Prolog predicates. It is optional, but recomended.
- `PrologTool` wraps the Prolog script with the interfase to LangChain/LangGraph. I supports all the methods of the `Tool` class and tracing capabilities of LangSmith.

In [1]:
#!pip install langchain-prolog

from langchain_prolog import PrologConfig, PrologRunnable, PrologTool

TEST_SCRIPT = "family.pl"

In [2]:
schema = PrologRunnable.create_schema("parent", ["man", "woman", "child"])
config = PrologConfig(
    rules_file=TEST_SCRIPT,
    query_schema=schema,
)
prolog_tool = PrologTool(
    prolog_config=config,
    name="family_query",
    description="""
        Query family relationships using Prolog.
        parent(X, Y, Z) implies only that Z is a child of X and Y.
        Input must be a dictionary like:
        {
            'man': 'richard',
            'woman': 'valery',
            'child': None,
        
        }
        Use 'None' to indicate a variable that need to be instantiated by Prolog
        The query will return:
            - 'True': if the relationship 'child' is a child of 'men' and 'women' holds.
            - 'False' if the relationship 'child' is a child of 'man' and 'woman' does not holds.
            - A list of dictionaries with all the answers to the Prolog query
        Do not use double quotes.
    """,
)

### Invocation

If a schema is defined, we can pass a dictionary using the names of the parameters in the schema as the keys in the dictionary. The values can represent Prolog variables (uppercase first letter) or strings (lower case first letter). A `None` value is interpreted as a variable and replaced with the key capitalized:

In [3]:
prolog_tool.invoke(
    {
        'man': 'john',
        'woman': None,
        'child': None
    }
)

[{'Woman': 'bianca', 'Child': 'mary'}, {'Woman': 'bianca', 'Child': 'michael'}]

There are three diferent ways to use a PrologTool to query Prolog:

### 1) Using a Prolog tool with an LLM and function calling

In [4]:
#!pip install python-dotenv

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), override=True)

#!pip install langchain langchain-openai

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

To use the tool, bind it to the LLM model:

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([prolog_tool])

and then query the model:

In [6]:
query = "Who are John's children?"
messages = [HumanMessage(query)]
response = llm_with_tools.invoke(messages)

The LLM will respond with a tool call request:

In [7]:
messages.append(response)
response.tool_calls[0]

{'name': 'family_query',
 'args': {'man': 'john', 'woman': None, 'child': None},
 'id': 'call_WshVu3c3rnqk1muTLUe3gBdp',
 'type': 'tool_call'}

The tool takes this request and queries the Prolog database:

In [8]:
tool_msg = prolog_tool.invoke(response.tool_calls[0])

The tool returns a list with all the solutions for the query:

In [9]:
messages.append(tool_msg)
tool_msg

ToolMessage(content='[{"Woman": "bianca", "Child": "mary"}, {"Woman": "bianca", "Child": "michael"}]', name='family_query', tool_call_id='call_WshVu3c3rnqk1muTLUe3gBdp')

That we then pass to the LLM, and the LLM answers the original query using the tool response:

In [10]:
answer = llm_with_tools.invoke(messages)
print(answer.content)

John has two children: Mary and Michael, with Bianca.


### 2) Using a Prolog tool with a LangChain agent
To use the prolog tool with an agent, pass it to the agent's constructor:

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
tools = [prolog_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

The agent takes the query and use the Prolog tool if needed:

In [12]:
answer = agent_executor.invoke({"input": "Who are John's children?"})
print(answer["output"])

John has two children: Mary and Michael. Their mother is Bianca.


### 3) Using a Prolog tool with a LangGrap agent

To use the prolog tool with an agent, pass it to the agent's constructor:

In [13]:
#!pip install langgraph 

from langgraph.prebuilt import create_react_agent

lg_agent = create_react_agent(llm, [prolog_tool])

The agent takes the query and use the Prolog tool if needed:

In [14]:
messages =lg_agent.invoke({"messages": [("human", query)]})

Then the agent receives​ the tool response and generates the answer:

In [15]:
messages["messages"][-1].pretty_print()

================================== Ai Message ==================================

John has two children: Mary and Michael, with Bianca as their mother.


### API reference

See https://langchain-prolog.readthedocs.io/en/latest/modules.html for detail.